![image](https://raw.githubusercontent.com/IBM/watsonx-ai-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `ibm/granite-3-3-8b-instruct` to analyze car rental customer satisfaction from text

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support of text sentiment analysis in watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.12.


## Learning goal

The goal of this notebook is to demonstrate how to use `ibm/granite-3-3-8b-instruct` model to analyze customer satisfaction from text.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Data loading](#data)
- [Foundation Models on watsonx](#models)
- [Model testing](#predict)
- [Score](#score)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials


### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U wget | tail -n 1
%pip install -U datasets | tail -n 1
%pip install -U "scikit-learn==1.6.1" | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [2]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

### Working with projects

First of all, you need to create a project that will be used for your work. If you do not have project already created follow bellow steps.

- Open IBM Cloud Pak main page
- Click all projects
- Create an empty project
- Copy `project_id` from url and paste it below

**Action**: Assign project ID below

In [4]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

#### Create `APIClient` instance

In [5]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials, project_id)

<a id="data"></a>
## Data loading

Download the `car_rental_training_data` dataset. The dataset provides insight about customers opinions on car rental. It has a label that consists of values: unsatisfied, satisfied.

In [6]:
import wget
import pandas as pd

filename = "car_rental_training_data.csv"
url = "https://raw.githubusercontent.com/IBM/watsonx-ai-samples/master/cpd5.2/data/cars-4-you/car_rental_training_data.csv"

if not os.path.isfile(filename):
    wget.download(url, out=filename)

df = pd.read_csv("car_rental_training_data.csv", sep=";")
data = df[["Customer_Service", "Satisfaction"]]

Examine downloaded data.

In [7]:
data.head()

,Customer_Service,Satisfaction
0,I thought the representative handled the initi...,0
1,I have had a few recent rentals that have take...,0
2,car cost more because I didn't pay when I rese...,0
3,I didn't get the car I was told would be avail...,0
4,If there was not a desired vehicle available t...,1


Prepare train and test sets.

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
comments = list(test.Customer_Service)
satisfaction = list(test.Satisfaction)

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

In [9]:
for model in client.foundation_models.TextModels:
    print(f"- {model}")

- ibm/granite-3-3-8b-instruct


You need to specify `model_id` that will be used for inferencing:

In [10]:
model_id = client.foundation_models.TextModels.GRANITE_3_3_8B_INSTRUCT

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to <a href="https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#metanames.GenTextParamsMetaNames" target="_blank" rel="noopener no referrer">documentation</a>.

In [11]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.MAX_NEW_TOKENS: 1,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.REPETITION_PENALTY: 1,
}

### Initialize the model
Initialize the `ModelInference` class with previous set params.

In [12]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id, params=parameters, credentials=credentials, project_id=project_id
)

### Model's details

In [13]:
model.get_details()

{'model_id': 'ibm/granite-3-3-8b-instruct',
 'label': 'granite-3-3-8b-instruct',
 'provider': 'IBM',
 'source': 'IBM',
 'functions': [{'id': 'autoai_rag'},
  {'id': 'multilingual'},
  {'id': 'text_chat'},
  {'id': 'text_generation'}],
 'short_description': 'Granite-3.3-8b-Instruct is an IBM-trained, dense decoder-only models, which is particularly well-suited for generative tasks.',
 'long_description': 'Granite-3.3-8b-Instruct is designed to be used for a wide range of generative and non-generative tasks with appropriate prompt engineering. It employs a GPT-style decoder-only architecture, with additional innovations from IBM Research and the open community.',
 'input_tier': 'class_12',
 'output_tier': 'class_12',
 'number_params': '8b',
 'min_shot_size': 1,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'code',
  'extraction',
  'translation',
  'function_calling'],
 'tasks': [{'id': 'question_answering

<a id="predict"></a>
## Analyze the satisfaction

#### Prepare prompt and generate text

In [14]:
instruction = """Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no"""

In [15]:
prompt1 = "\n".join([instruction, "Comment:" + comments[2], "Satisfied:"])
print(prompt1)

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no
Comment:Customer service was fine.  We didn't care for the off site location.  Would not use that car rental company at that airport again.
Satisfied:


Analyze the sentiment for a sample of zero-shot input from the test set.

In [16]:
print(model.generate_text(prompt=prompt1))

no


### Calculate the accuracy

In [17]:
sample_size = 10
prompts_batch = [
    "\n".join([instruction, "Comment:" + comment, "Satisfied:"])
    for comment in comments[:10]
]
results = model.generate_text(prompt=prompts_batch)

In [18]:
print(prompts_batch[0])

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no
Comment:the rep was friendly but it was so loud in there that I could not hear what she was saying. I HATE having to walk across a big lot with all of my bags in search of my car which is always in the furthest corner.
Satisfied:


<a id="score"></a>
## Score the model

In [19]:
from sklearn.metrics import accuracy_score

label_map = {0: "no", 1: "yes"}
y_true = [label_map[sat] for sat in satisfaction][:sample_size]

print("accuracy_score", accuracy_score(y_true, results))

accuracy_score 0.9


In [20]:
print("true", y_true)
print("pred", results)

true ['no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'no', 'no']
pred ['no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no']


<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!
 
 You learned how to analyze car rental customer satisfaction with watsonx.ai foundation model.
 
 Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Mateusz Szewczyk**, Software Engineer at watsonx.ai.

**Lukasz Cmielowski**, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.